# Capstone in Mathematics Final Project
## Logistic Regression -- Patrick Thornton
## Not intended to be graded, just for those interested

In [46]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import model_selection

In [47]:
def read_data(feature):
    x_data = pd.read_csv(feature)
    x = x_data.values
    return x

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [48]:
train.rename(columns={'weathersit':'weather'},inplace=True)
train = train.drop(['dteday'], axis=1)
train['season'] = train.season.astype('category')
train['holiday'] = train.holiday.astype('category')
train['weather'] = train.weather.astype('category')

test.rename(columns={'weathersit':'weather'},inplace=True)
test = test.drop(['dteday'], axis=1)
test['season'] = test.season.astype('category')
test['holiday'] = test.holiday.astype('category')
test['weather'] = test.weather.astype('category')

## One hot encoder
### label is 0 -> [1 0 0 0 0 0 0 0 0]
### label is 3 -> [0 0 0 1 0 0 0 0 0]

In [49]:
def one_hot_encoder(df, column):       
    df = pd.concat([df, pd.get_dummies(df[column], prefix=column, drop_first=True)],axis=1)
    df = df.drop([column], axis=1)
    return df

train_ohe = train
test_ohe = test

cats = ['season','holiday','weather']
for cat in cats:
    train_ohe = one_hot_encoder(train_ohe, cat)
    test_ohe = one_hot_encoder(test_ohe, cat)

In [50]:
y_train =  train_ohe['cnt']
X_train = train_ohe.drop(['cnt'], axis=1)
y_test =  test_ohe['cnt']
X_test = test_ohe.drop(['cnt'], axis=1)

In [51]:
y_train = pd.DataFrame(y_train, columns = ['cnt'])
y_test = pd.DataFrame(y_test, columns = ['cnt'])

## Normalize labels

In [52]:
'''
    normalize the features
    Using normal distribution
'''
def get_mean_variance(X):
    mean = np.mean(X, axis=0) # axis=0: taking means along the
    # vertical line (column)
    # (sum(x_i-\mu)^2)/N
    X_temp = X - mean #
    X_temp_entrypointwise = X_temp*X_temp
    variance = np.mean(X_temp_entrypointwise, axis=0) #axis=0: 
    # taking means along the vertical line (column)
    return mean, variance
    
def normalize_features(X_train, X_test):
    mean, variance = get_mean_variance(X_train)
    variance += 1e-15
    ''' transform the feature '''
    X_train_norm = (X_train - mean)/(np.sqrt(variance))
    #math.sqrt doesnot work for numpy
    X_test_norm = (X_test - mean)/np.sqrt(variance)
    return X_train_norm, X_test_norm

X_train_norm, X_test_norm = normalize_features(X_train, X_test)

In [53]:
import math
y_train = y_train.transform(lambda x: (x/250)).astype(int)
y_test = y_test.transform(lambda x: (x/250)).astype(int)


##  Add Bias

In [54]:
def add_column_one(X):
    '''
         convert X -> [1 X]
    '''
    # add  column of ones
    ones = np.ones(X.shape[0])
    ones = ones.reshape(-1, 1)
    X_new = np.append(ones, X, axis=1)

    return X_new

X_train_norm_new = add_column_one(X_train_norm)
X_test_norm_new = add_column_one(X_test_norm)

## Predictor

In [55]:
def predictor(X, c):
    ''' sigmoid function '''
    return 1.0/(1.0 + np.exp(-X.dot(c)))

## Loss function

In [56]:
def loss(y_exact, y_pred):
    return (-y_exact.T.dot(np.log(y_pred+1e-15))- (1.0 - y_exact).T.dot(np.log(1-y_pred+1e-15)))/float(len(y_exact))

## Gradient Descent

In [57]:
def gradient_descent(X, y, epochs=1000, learning_rate=0.0001):
    '''
        Input
        -----
        X: training features (normalized and having bias)
        y: labels
        
        output
        ------
        c: optimal coeffs
        loss_history
    '''
    loss_history = [0]*epochs
    c_dim = X.shape[1]
    n_samples = X.shape[0]
    c = np.ones((c_dim, 1))
    for epoch in range(epochs):
        y_pred = predictor(X, c)
        
        loss_history[epoch] = loss(y_pred, y).ravel()[0] # (2D) (1,1) -> 1D
                                                               # [5] -> 5
       
        XT = X.T
        gradient = XT.dot(y_pred-y)/float(n_samples)   
        # updating coeffs upon the gradient change
        c = c - learning_rate*gradient
    return c, loss_history

def plot_loss(loss):
    import matplotlib  # import package
    import matplotlib.pyplot as plt # import library pytlot and change its name to plt
    %matplotlib inline  
    plt.xlabel('# of epochs')
    plt.ylabel('Loss')
    plt.plot(loss)
    plt.show()  

In [58]:
def one_label_encoder(y_train, y_test):
    ''' convert label to a vector under one-hot-code fashion '''
    from sklearn import preprocessing
    lb = preprocessing.LabelBinarizer()
    y_train = np.array(y_train, dtype=np.float32)
    y_test = np.array(y_test, dtype=np.float32)
    lb.fit(y_train)
    y_train_ohe = lb.transform(y_train)
    y_test_ohe = lb.transform(y_test)
    return y_train_ohe, y_test_ohe
# label is 0 -> [1 0 0 0 0 0 0 0 0]
# label is 3 -> [0 0 0 1 0 0 0 0 0]
y_train_ohe, y_test_ohe = one_label_encoder(y_train, y_test)


## Multi label train
## One vs. the rest

In [59]:
def multilabel_train(X, y):# y_train: one_hot_encoder labels
    c_list = []
    print(y.shape)
    for i in range(y.shape[1]): # 3 columns 
        y_one_column = y[:, i].reshape(-1, 1) # pick ith columns
        X = np.array(X,dtype=np.float32)
        c_one_column, loss_history = gradient_descent(X, y_one_column, epochs=100, learning_rate=0.9)
        #plot_loss(loss_history)
        c_list.append(c_one_column)
    return c_list
c_list = multilabel_train(X_train_norm_new, y_train_ohe)

(5000, 4)


## Prediction

In [60]:
def multilabel_prediction(c_list, X):
    i = 0
    X = np.array(X,dtype=np.float32)
    for c in c_list:
        probability = predictor(X, c)
        # probabily of one column
        if i == 0:
            probability_matrix = probability
        else:
            # combine all decision columns to form a matrix
            probability_matrix = np.concatenate(
                              (probability_matrix, probability),
                               axis=1)
        i += 1
    labels = []
    for i in range(0,10000):
        labels.append(i)
    n_samples = X.shape[0]
    # find which index gives us the highest probability
    y_pred = np.zeros(n_samples, dtype=int)
    for i in range(n_samples):
        y_pred[i] = labels[np.argmax(probability_matrix[i,:])]
    return y_pred

y_pred = multilabel_prediction(c_list, X_test_norm_new)

## Accuracy

In [74]:
def accuracy(ypred, yexact):
    p = []
    y = np.asarray(yexact)
    for i in range(len(ypred)):
        if ypred[i] == y[i][0]:
            p.append(1)
        else:
            p.append(0)
            
    return np.sum(p)/float(len(yexact))
print("Accuracy = %.4f" % accuracy(y_pred, y_test))

Accuracy = 0.7220
